In [ ]:
import os
import tensorflow as tf
import numpy as np
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt
from scipy import ndimage as nd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
#session = tf.compat.v1.Session(config=config)

In [ ]:

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
except:
    print('whss')
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [ ]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(
#   physical_devices[0],
#   [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000),
#   tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])

In [ ]:
# print(tf.config.list_physical_devices('GPU'))

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#         print(e)

In [ ]:

    seed = 42
    np.random.seed = seed


    IMG_WIDTH = 512
    IMG_HEIGHT = 1024
    IMG_CHANNELS = 3

    TRAIN_PATH = '../data_files/files_half/traindata/'
    TRAIN_PATH_Y = '../data_files/files_half/groundtruth/'
    TEST_PATH = '../data_files/files_half/testdata/'
    TEST_PATH_Y = '../data_files/files_half/testy/'

    train_files=[]
    for file in os.listdir("../data_files/files_half/traindata"):
        if file.endswith(".png"):
            train_files.append(file)

    
    train_files_y=[]
    for file in os.listdir("../data_files/files_half/groundtruth"):
        if file.endswith(".png"):
            train_files_y.append(file)

#test_files=[]
#for file in os.listdir("/content/files3_512/testdata"):
#    if file.endswith(".png"):
#        test_files.append(file)

    X_train = np.zeros((len(train_files),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS), dtype = np.uint8)
    Y_train = np.zeros((len(train_files),IMG_HEIGHT,IMG_WIDTH,1), dtype = np.bool)

#train_files.sort()
#train_files_y.sort()
    i=0
    for file in train_files:
        path = TRAIN_PATH + file
        img = cv2.imread(path)[:,:,:IMG_CHANNELS]  
    
        #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        #img = np.expand_dims(img,axis=2)
        X_train[i]=img
        path=''
        path = TRAIN_PATH_Y + file
        img = cv2.imread(path)[:,:,:1]  
    #img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #img = np.expand_dims(img,axis=2)
        Y_train[i]=img
        i=i+1

    i=0

#X_test = np.zeros((len(test_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
#sizes_test = []
#for file in test_files:
#    path = TEST_PATH + file
#    img = cv2.imread(path)[:,:,:IMG_CHANNELS]
#    sizes_test.append([img.shape[0], img.shape[1]])
    #img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #img = np.expand_dims(img,axis=2)
#    X_test[i] = img
#    i=i+1




In [ ]:
X_train.shape

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT,IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [ ]:

#Contraction path

nfilters=1024

c1 = tf.keras.layers.Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(int(nfilters*2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(int(nfilters*2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)


#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(int(nfilters), (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(int(nfilters/2), (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(int(nfilters/4), (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(int(nfilters/8), (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)


outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)


In [ ]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon()) 

model2 = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])
model2.optimizer.lr=0.0001


#model.save('trained_model.hdf5')


In [ ]:
model2.load_weights('/home/meng/Gany_Lab/TEM_processing/MRCFILE/model_1024_prec_6.hdf5')

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('unet_membrane.hdf5',verbose = 1, save_best_only=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 10,monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')]

results = model2.fit(X_train,Y_train,validation_split=0.1,batch_size=4,epochs=100,callbacks=callbacks)
#model2.save('channel_1_1024')

In [ ]:
 model2.save_weights('channel_1_1024.hdf5',save_format='hdf5')

In [ ]:
model2.save('channel_1_1024',save_format='h5')

In [ ]:
!zip -r model_contour.zip contour_model_256/

In [ ]:
def f1(y_true, y_pred):
    return 1
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon()) 

contour_model = tf.keras.models.load_model('contour_model_256', custom_objects = {"f1": f1, "dice_coefficient": dice_coefficient})

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
model2 = tf.keras.models.load_model('contour_model_512')

In [ ]:
model=model2

In [ ]:


score, acc = model.evaluate(xt,yt
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
!mkdir testimgs

In [ ]:
!mkdir predimgs

In [ ]:
preds_test = model2.predict(xt, verbose=1)

In [ ]:
preds_test_t= (preds_test > 0.1).astype(np.uint8)

In [ ]:
ytt = (yt).astype(np.uint8)

In [ ]:
from sklearn.metrics import jaccard_score
jscores = []
for i in range(0,200):
 jscores.append(jaccard_score(np.squeeze(ytt[i]).reshape(-1), np.squeeze(preds_test_t[i]).reshape(-1)))

In [ ]:
np.mean(jscores)

In [ ]:

preds_test = model2.predict(X_test, verbose=1)

preds_test= (preds_test > 0.4).astype(np.uint8)

plt.imsave('img1.png',(np.squeeze(preds_test[0])),cmap='gray')
plt.imsave('img2.png',(np.squeeze(preds_test[1])),cmap='gray')
plt.imsave('img3.png',(np.squeeze(preds_test[2])),cmap='gray')
plt.imsave('img4.png',(np.squeeze(preds_test[3])),cmap='gray')
plt.imsave('img5.png',(np.squeeze(preds_test[4])),cmap='gray')
plt.imsave('img6.png',(np.squeeze(preds_test[5])),cmap='gray')


In [ ]:
i=0
test_files=[]
TEST_PATH="/content/files512/testdata/"
for file in os.listdir("/content/files512/testdata/"):
    if file.endswith(".png"):
        test_files.append(file)
X_test = np.zeros((len(test_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
for file in test_files:
    path = TEST_PATH + file
    img = cv2.imread(path)[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #img = np.expand_dims(img,axis=2)
    X_test[i] = img
    i=i+1

In [ ]:
i=0

X_test = np.zeros((len(test_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
for file in test_files:
    path = TEST_PATH + file
    img = cv2.imread(path)[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #img = np.expand_dims(img,axis=2)
    X_test[i] = img
    i=i+1


In [ ]:
test_files

In [ ]:
preds_test = model2.predict(X_test, verbose=1)


In [ ]:

preds_test_t= (preds_test>0.1).astype(np.uint8)
plt.imsave('img1.png',(np.squeeze(preds_test_t[0])),cmap='gray')
plt.imsave('img2.png',(np.squeeze(preds_test_t[1])),cmap='gray')
plt.imsave('img3.png',(np.squeeze(preds_test_t[2])),cmap='gray')
plt.imsave('img4.png',(np.squeeze(preds_test_t[3])),cmap='gray')

In [ ]:
!zip -r model.zip /content/trained_model_512/


In [ ]:

from tensorflow.keras.losses import binary_crossentropy
from keras.models import model_from_json

from keras.layers import Input, Conv2D, Reshape
from keras.models import Model

In [ ]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon()) 

In [ ]:
def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - tf.math.log(dice_coefficient(y_true, y_pred) + tf.keras.backend.epsilon())

In [ ]:

model3 = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model3.compile(optimizer='sgd', loss=loss, metrics=[dice_coefficient])
model3.optimizer.lr=0.0001


#model.save('trained_model.hdf5')


In [ ]:

model3.fit(X_train,Y_train,batch_size=32,epochs=30,validation_split=0.1)